# Spaceship Titanic Science Solutions
[Notebook used 
](https://www.kaggle.com/code/startupsci/titanic-data-science-solutions)  thanks to @[Manav Sehgal](https://www.kaggle.com/startupsci)
# Workflow stages

The competition solution workflow goes through seven stages described in the Data Science Solutions book.

1. Question or problem definition.
2. Acquire training and testing data.
3. Wrangle, prepare, cleanse the data.
4. Analyze, identify patterns, and explore the data.
5. Model, predict and solve the problem.
6. Visualize, report, and present the problem solving steps and final solution.
7. Supply or submit the results.

The workflow indicates general sequence of how each stage may follow the other. However there are use cases with exceptions.

- We may combine mulitple workflow stages. We may analyze by visualizing data.
- Perform a stage earlier than indicated. We may analyze data before and after wrangling.
- Perform a stage multiple times in our workflow. Visualize stage may be used multiple times.
- Drop a stage altogether. We may not need supply stage to productize or service enable our dataset for a competition.


# Question and problem definition

Competition sites like Kaggle define the problem to solve or questions to ask while providing the datasets for training your data science model and testing the model results against a test dataset. The question or problem definition To help rescue crews and retrieve the lost passengers, you are challenged to predict which passengers were transported by the anomaly using records recovered from the spaceship’s damaged computer system., competition is [described here at Kaggle](https://www.kaggle.com/competitions/spaceship-titanic).

> Knowing from a training set of samples listing passengers who Transported or did not the Titanic disaster, can our model determine based on a given test dataset not containing the survival information, if these passengers in the test dataset Transported or not.

We may also needs to develop some early understanding about the domain of our problem. This is described on the [Kaggle competition description page here](https://www.kaggle.com/competitions/spaceship-titanic). 

# Workflow goals

The data science solutions workflow solves for seven major goals.

**Classifying.** We may needs to classify or categorize our samples. We may also want to understand the implications or correlation of different classes with our solution goal.

**Correlating.** One can approach the problem based on available features within the training dataset. Which features within the dataset contribute significantly to our solution goal? Statistically speaking is there a [correlation](https://en.wikiversity.org/wiki/Correlation) among a feature and solution goal? As the feature values change does the solution state change as well, and visa-versa? This can be tested both for numerical and categorical features in the given dataset. We may also want to determine correlation among features other than survival for subsequent goals and workflow stages. Correlating certain features may help in creating, completing, or correcting features.

**Converting.** For modeling stage, one needs to prepare the data. Depending on the choice of model algorithm one may require all features to be converted to numerical equivalent values. So for instance converting text categorical values to numeric values.

**Completing.** Data preparation may also require us to estimate any missing values within a feature. Model algorithms may work best when there are no missing values.

**Correcting.** We may also analyze the given training dataset for errors or possibly innacurate values within features and try to corrent these values or exclude the samples containing the errors. One way to do this is to detect any outliers among our samples or features. We may also completely discard a feature if it is not contribting to the analysis or may significantly skew the results.

**Creating.** Can might create new features based on an existing feature or a set of features, such that the new feature follows the correlation, conversion, completeness goals.

**Charting.** How to select the right visualization plots and charts depending on nature of the data and the solution goals.

In [ ]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

# Acquire data

The Python Pandas packages helps us work with our datasets. We start by acquiring the training and testing datasets into Pandas DataFrames. We also combine these datasets to run certain operations on both datasets together.

In [ ]:
PATH = "/kaggle/input/spaceship-titanic/"
!ls {PATH}

In [ ]:
train_df = pd.read_csv(f'{PATH}train.csv', low_memory=False )
test_df = pd.read_csv(f'{PATH}test.csv', low_memory=False )

## Analyze by describing data

Pandas also helps describe the datasets answering following questions early in our project.

**Which features are available in the dataset?**

Noting the feature names for directly manipulating or analyzing these. These feature names are described on the [Kaggle data page here](https://www.kaggle.com/competitions/spaceship-titanic/data).

In [ ]:
print(train_df.columns.values)

**Which features are categorical?**

These values classify the samples into sets of similar samples. Within categorical features are the values nominal, ordinal, ratio, or interval based? Among other things this helps us select the appropriate plots for visualization.

- Categorical: HomePlanet, CryoSleep, Cabin,Destination and VIP.

**Which features are numerical?**

Which features are numerical? These values change from sample to sample. Within numerical features are the values discrete, continuous, or timeseries based? Among other things this helps us select the appropriate plots for visualization.

- Continous: 'Age' 'RoomService' 'FoodCourt' 'ShoppingMall' 'Spa' 'VRDeck'.

In [ ]:
# preview the data
train_df.head()

**Which features may contain errors or typos?**

This is harder to review for a large dataset, however reviewing a few samples from a smaller dataset may just tell us outright, which features may require correcting.

- Name feature may contain errors or typos as there are several ways used to describe a name including titles, round brackets, and quotes used for alternative or short names.

In [ ]:
train_df.tail()

**Which features contain blank, null or empty values?**

These will require correcting.

- Cabin  Embarked features contain a number of null values in that order for the training and test dataset.
- Age features contain a number of null values in that order for the training and test dataset. 
**What are the data types for various features?**

Helping us during converting goal.

- Seven features are object or string. Six are float. as shown below

In [ ]:
round(train_df.isnull().sum()*100/len(train_df),2).sort_values(ascending=False)

In [ ]:
round(test_df.isnull().sum()*100/len(test_df),2).sort_values(ascending=False)

**What is the distribution of numerical feature values across the samples?**

This helps us determine, among other early insights, how representative is the training dataset of the actual problem domain.

- Total samples are 437 or 50% of the actual number of passengers Transported while other 50% are not.
- Transported is a categorical feature with 0 or 1 values.
- 2% of passengers are VIP.
- 68% of passengers destination is TRAPPIST-1e 
- 55 of passengers destination is Cancri e
- 38 of Passengers within age range 38-75


In [ ]:
train_df.describe()

**What is the distribution of categorical features?**

- Destination variable as three possible values with 68% TRAPPIST-1e and 21% of 55 Cancri e and 11% for others.
- Cabin values have several dupicates across samples about (2132). Alternatively several passengers shared a cabin.


In [ ]:
train_df.Cabin.duplicated().sum() ,

In [ ]:
train_df.describe(include=['O'])

In [ ]:
train_df.nunique().sort_values(ascending=False)

# Assumtions based on data analysis

We arrive at following assumptions based on data analysis done so far. We may validate these assumptions further before taking appropriate actions.

**Correlating.**

We want to know how well does each feature correlate with Transported. We want to do this early in our project and match these quick correlations with modelled correlations later in the project.

**Completing.**

1. We may need to complete Age feature as it is definitely correlated to Transported.
2. We may need to complete the Cabin feature as it may also correlate with Transported or another important feature.

**Correcting.**

1. We will handel null valuese over all featuers.
2. we will drop PassengerId and Name featuers.
3. we will split Cabin featuer to three valuese 'deck', 'num','side', then we will drop the orignal column.

**Creating.**

1. We may need to create new feature for Age bands. This turns a continous numerical feature into an ordinal categorical feature.

**Classifying.**

We may also add to our assumptions based on the problem description noted earlier.

1. The upper-class passengers (VIP=1) were more likely to have Transported. 

# Analyze by pivoting features

To confirm some of our observations and assumptions, we can quickly analyze our feature correlations by pivoting features against each other. We can only do so at this stage for features which do not have any empty values. It also makes sense doing so only for features which are categorical ( HomePlanet), CryoSleep, Cabin, Destination, VIP.

- **HomePlanet and Destination** We observe significant correlation (>0.65)  Transported (classifying #3). We decide to include this feature in our model.
- **CryoSleep** We observe significant correlation (>0.8) among CryoSleep=1 and Transported (classifying #3). We decide to include this feature in our model.
- **VIP** We confirm the observation during problem definition that VIP=false had very high Transported rate at 50% (classifying #1).
- **Cabin** These features have zero correlation for certain values. It may be best to derive a feature or a set of features from these individual features, we can spilt it inot three other featuers 'deck', 'num','side' (creating #1).

Categorical: HomePlanet, CryoSleep, Cabin,Destination and VIP.

In [ ]:
train_df[['HomePlanet', 'Transported']].groupby(['HomePlanet'], as_index=False).mean().sort_values(by='Transported', ascending=False)

In [ ]:
train_df[['CryoSleep', 'Transported']].groupby(['CryoSleep'], as_index=False).mean().sort_values(by='Transported', ascending=False)

In [ ]:
train_df[['Cabin', 'Transported']].groupby(['Cabin'], as_index=False).mean().sort_values(by='Transported', ascending=False)

In [ ]:
train_df[['Destination', 'Transported']].groupby(['Destination'], as_index=False).mean().sort_values(by='Transported', ascending=False)

In [ ]:
train_df[['VIP', 'Transported']].groupby(['VIP'], as_index=False).mean().sort_values(by='Transported', ascending=False)

# Analyze by visualizing data

Now we can continue confirming some of our assumptions using visualizations for analyzing the data.

### Correlating numerical features

Let us start by understanding correlations between numerical features and our solution goal (Transported).

A histogram chart is useful for analyzing continous numerical variables like Age where banding or ranges will help identify useful patterns. The histogram can indicate distribution of samples using automatically defined bins or equally ranged bands. This helps us answer questions relating to specific bands (Did infants have better survival rate?)

Note that x-axis in historgram visualizations represents the count of samples or passengers.

**Observations.**

- Age <=40 had high Transport rate.
- Oldest passengers (Age = 80) Transported.
- Large number of 20-25 year olds did not Transported.
- Most passengers are in 15-35 age range.

**Decisions.**

This simple analysis confirms our assumptions as decisions for subsequent workflow stages.

- We should consider Age (our assumption classifying #2) in our model training.
- Complete the Age feature for null values (completing #1).
- We should band age groups (creating #3).

In [ ]:
g = sns.FacetGrid(train_df[train_df['Age'] > 0], col='Transported')
g.map(plt.hist, 'Age', bins=20)

### Correlating numerical and ordinal features

We can combine multiple features for identifying correlations using a single plot. This can be done with numerical and categorical features which have numeric values.

**Observations.**

- Destination=TRAPPIST-1e had most passengers, however most did not transported. Confirms our classifying assumption #2.
- Infant passengers in Destination=PSO J318.5-22 and Destination=55 Cancri e mostly transported. Further qualifies our classifying assumption #2.
- Most passengers in Destination=TRAPPIST-1e survived. Confirms our classifying assumption #3.
- Destination varies in terms of Age distribution of passengers.

**Decisions.**

- Consider Destination for model training.

Continous: 'Age' 'RoomService' 'FoodCourt' 'ShoppingMall' 'Spa' 'VRDeck'.

In [ ]:
grid = sns.FacetGrid(train_df, col='Transported', row='Destination', height=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend();

### Correlating categorical features

Now we can correlate categorical features with our solution goal.

**Observations.**

- Non VIP passengers had much better Transported rate than VIP. Confirms classifying (#1).
- VIP passengers had better Transported rate in CryoSleep=True when compared with CryoSleep=False. Completing (#2).
**Decisions.**

- Add VIP feature to model training.
- Complete and add CryoSleep feature to model training.

Categorical: HomePlanet, CryoSleep, Cabin,Destination and VIP.

In [ ]:
train_df.head()

In [ ]:
grid = sns.FacetGrid(train_df, row='VIP', col='Transported', size=2.2, aspect=1.6)
grid.map(sns.barplot, 'CryoSleep', 'Destination', alpha=.5, ci=None)
grid.add_legend()

# Wrangle data

We have collected several assumptions and decisions regarding our datasets and solution requirements. So far we did not have to change a single feature or value to arrive at these. Let us now execute our decisions and assumptions for correcting, creating, and completing goals.

### Correcting by dropping features

This is a good starting goal to execute. By dropping features we are dealing with fewer data points. Speeds up our notebook and eases the analysis.

Based on our assumptions and decisions we want to drop the Name (correcting #2) and PassengerId (correcting #1) features.

Note that where applicable we perform operations on both training and testing datasets together to stay consistent.

Lets drop  the passengerid and Name from train and test data, (Correcting1,Correcting2)

In [ ]:
train_df=train_df.drop(['PassengerId','Name'],axis=1)
test_df=test_df.drop(['PassengerId','Name'],axis=1)


### Creating new feature extracting from existing

In the following code we extract 'deck', 'num','side' from cabin feature using split method.then we will drop the the oreginal feature (creating #1)

In [ ]:
train_df[['deck', 'num','side']] = train_df['Cabin'].str.split('/', expand=True)
train_df=train_df.drop(['Cabin'],axis=1)
train_df.head()

In [ ]:
test_df[['deck', 'num','side']] = test_df['Cabin'].str.split('/', expand=True)
test_df=test_df.drop(['Cabin'],axis=1)
test_df.head()

In [ ]:
train_df.nunique().sort_values(ascending=False)

In [ ]:
train_df['deck'].value_counts()

In [ ]:
train_df['deck'].unique().tolist()


### Converting a categorical feature

Now we can convert features which contain strings to numerical values. This is required by most model algorithms. Doing so will also help us in achieving the feature completing goal.

We'll replace categories with their numeric codes, handle missing continuous values, and split the Cabin variable into a separate variable (creating #1).


In [ ]:
train_df['deck']=train_df['deck'].replace({'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6, 'T':7})
train_df['deck'].value_counts()

In [ ]:
test_df['deck']=test_df['deck'].replace({'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6, 'T':7})
test_df['deck'].value_counts()

In [ ]:
train_df.head()

also, we need to convert side features to numerical values. let's see what unique values do we have?

In [ ]:
train_df['side'].unique().tolist()

In [ ]:
train_df['side']=train_df['side'].replace({'P':0, 'S':1})
train_df['side'].value_counts()

In [ ]:
test_df['side']=test_df['deck'].replace({'P':0, 'S':1})
test_df['side'].value_counts()

In [ ]:
train_df.head()

We need to convert HomePlanet features to numerical values. let's see what unique values do we have?

In [ ]:
train_df['HomePlanet'].unique().tolist()

In [ ]:
train_df['HomePlanet']=train_df['HomePlanet'].replace({'Europa':0, 'Earth':1, 'Mars':1})
train_df['HomePlanet'].value_counts()

In [ ]:
test_df['HomePlanet']=test_df['HomePlanet'].replace({'Europa':0, 'Earth':1, 'Mars':1})
test_df['HomePlanet'].value_counts()

We need to convert Destination features to numerical values. let's see what unique values do we have?

In [ ]:
train_df['Destination'].unique().tolist()

In [ ]:
train_df['Destination']=train_df['Destination'].replace({'TRAPPIST-1e':0, 'PSO J318.5-22':1, '55 Cancri e':2})
train_df['Destination'].value_counts()

In [ ]:
test_df['Destination']=test_df['Destination'].replace({'TRAPPIST-1e':0, 'PSO J318.5-22':1, '55 Cancri e':2})
test_df['Destination'].value_counts()

In [ ]:
train_df.head()

Lets convert CryoSleep & VIP feature to a column of integers 1 or 0.

In [ ]:
train_df[['CryoSleep', 'VIP','Transported']] = (train_df[['CryoSleep', 'VIP','Transported']] == True).astype(int)
test_df[['CryoSleep', 'VIP']] = (test_df[['CryoSleep', 'VIP']] == True).astype(int)
train_df.tail()


# Completing a numerical continuous feature

Before we prepare the data for modeling, we need to deal with the missing data. We’ll fill the missing values with an average value and then assign the results to data. This is a method of interpolation. The DataFrame.interpolate() method makes this simple.

This is a quick and simple method of dealing with missing values, and might not lead to the best performance of the model on new data. Handling missing values is an important part of the modeling process, where creativity and insight can make a big difference. This is another area where you can extend on this tutorial.


In [ ]:
nulls = pd.DataFrame(train_df.isnull().sum().sort_values(ascending=False)[:25])
nulls.columns = ['Null Count']
nulls.index.name = 'Feature'
nulls

Before we prepare the data for modeling, we need to deal with the missing data. We’ll fill the missing values with an average value and then assign the results to data. This is a method of interpolation. The DataFrame.interpolate() method makes this simple.

This is a quick and simple method of dealing with missing values, and might not lead to the best performance of the model on new data. Handling missing values is an important part of the modeling process, where creativity and insight can make a big difference. This is another area where you can extend on this tutorial.

In [ ]:
train_df = train_df.select_dtypes(include=[np.number]).interpolate().dropna()
test_df = test_df.select_dtypes(include=[np.number]).interpolate().dropna()

sum(train_df.isnull().sum() != 0)

## Model, predict and solve

Now we are ready to train a model and predict the required solution. There are 60+ predictive modelling algorithms to choose from. We must understand the type of problem and solution requirement to narrow down to a select few models which we can evaluate. Our problem is a classification and regression problem. We want to identify relationship between output (Transported or not) with other variables or features (Gender, CryoSleep, VIP...). We are also perfoming a category of machine learning which is called supervised learning as we are training our model with a given dataset. With these two criteria - Supervised Learning plus Classification and Regression, we can narrow down our choice of models to a few. These include:

- Logistic Regression
- KNN or k-Nearest Neighbors
- Support Vector Machines
- Naive Bayes classifier
- Decision Tree
- Random Forrest
- Perceptron
- Artificial neural network
- RVM or Relevance Vector Machine

In [ ]:
X_train = train_df.drop("Transported", axis=1)
Y_train = train_df["Transported"]
X_test  = test_df
X_train.shape, Y_train.shape, X_test.shape

Logistic Regression is a useful model to run early in the workflow. Logistic regression measures the relationship between the categorical dependent variable (feature) and one or more independent variables (features) by estimating probabilities using a logistic function, which is the cumulative logistic distribution. Reference [Wikipedia](https://en.wikipedia.org/wiki/Logistic_regression).

Note the confidence score generated by the model based on our training dataset.

In [ ]:
# Logistic Regression

logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

We can use Logistic Regression to validate our assumptions and decisions for feature creating and completing goals. This can be done by calculating the coefficient of the features in the decision function.

Positive coefficients increase the log-odds of the response (and thus increase the probability), and negative coefficients decrease the log-odds of the response (and thus decrease the probability).


In [ ]:
coeff_df = pd.DataFrame(train_df.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

Next we model using Support Vector Machines which are supervised learning models with associated learning algorithms that analyze data used for classification and regression analysis. Given a set of training samples, each marked as belonging to one or the other of **two categories**, an SVM training algorithm builds a model that assigns new test samples to one category or the other, making it a non-probabilistic binary linear classifier. Reference [Wikipedia](https://en.wikipedia.org/wiki/Support_vector_machine).

Note that the model generates a confidence score which is higher than Logistics Regression model.

In [ ]:
# Support Vector Machines

svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

In pattern recognition, the k-Nearest Neighbors algorithm (or k-NN for short) is a non-parametric method used for classification and regression. A sample is classified by a majority vote of its neighbors, with the sample being assigned to the class most common among its k nearest neighbors (k is a positive integer, typically small). If k = 1, then the object is simply assigned to the class of that single nearest neighbor. Reference [Wikipedia](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm).

KNN confidence score is better than Logistics Regression but worse than SVM.

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

In machine learning, naive Bayes classifiers are a family of simple probabilistic classifiers based on applying Bayes' theorem with strong (naive) independence assumptions between the features. Naive Bayes classifiers are highly scalable, requiring a number of parameters linear in the number of variables (features) in a learning problem. Reference [Wikipedia](https://en.wikipedia.org/wiki/Naive_Bayes_classifier).

The model generated confidence score is the lowest among the models evaluated so far.

In [ ]:
# Gaussian Naive Bayes

gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
acc_gaussian

The perceptron is an algorithm for supervised learning of binary classifiers (functions that can decide whether an input, represented by a vector of numbers, belongs to some specific class or not). It is a type of linear classifier, i.e. a classification algorithm that makes its predictions based on a linear predictor function combining a set of weights with the feature vector. The algorithm allows for online learning, in that it processes elements in the training set one at a time. Reference [Wikipedia](https://en.wikipedia.org/wiki/Perceptron).

In [ ]:
# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
acc_perceptron

In [ ]:
# Linear SVC

linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
acc_linear_svc

In [ ]:
# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
acc_sgd

This model uses a decision tree as a predictive model which maps features (tree branches) to conclusions about the target value (tree leaves). Tree models where the target variable can take a finite set of values are called classification trees; in these tree structures, leaves represent class labels and branches represent conjunctions of features that lead to those class labels. Decision trees where the target variable can take continuous values (typically real numbers) are called regression trees. Reference [Wikipedia](https://en.wikipedia.org/wiki/Decision_tree_learning).

The model confidence score is the highest among models evaluated so far.

In [ ]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

The next model Random Forests is one of the most popular. Random forests or random decision forests are an ensemble learning method for classification, regression and other tasks, that operate by constructing a multitude of decision trees (n_estimators=100) at training time and outputting the class that is the mode of the classes (classification) or mean prediction (regression) of the individual trees. Reference [Wikipedia](https://en.wikipedia.org/wiki/Random_forest).

The model confidence score is the highest among models evaluated so far. We decide to use this model's output (Y_pred) for creating our competition submission of results.

In [ ]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

train_df['deck'].unique().tolist()### Model evaluation

We can now rank our evaluation of all the models to choose the best one for our problem. While both Decision Tree and Random Forest score the same, we choose to use Random Forest as they correct for decision trees' habit of overfitting to their training set. 

In [ ]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree]})
models.sort_values(by='Score', ascending=False)

In [ ]:
sol=pd.read_csv('../input/spaceship-titanic/sample_submission.csv')
sol['Transported']=Y_pred
sol.head()
sol.to_csv('submission1226.csv',index=False)

# Thank you, Looking forward to your comments and upvotes